# Setting up my full dataframe

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

%matplotlib inline
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split

import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import WordNetLemmatizer

import gensim, logging, warnings
import gensim.corpora as corpora
from gensim import corpora, models
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df0 = pd.read_csv('data_files/scrape1.csv', index_col = 0)
df1 = pd.read_csv('data_files/scrape2.csv', index_col = 0)
df2 = pd.read_csv('data_files/scrape3.csv', index_col = 0)
df3 = pd.read_csv('data_files/scrape4.csv', index_col = 0)
df4 = pd.read_csv('data_files/scrape5.csv', index_col = 0)
df5 = pd.read_csv('data_files/scrape6.csv', index_col = 0)
df6 = pd.read_csv('data_files/scrape7.csv', index_col = 0)
df7 = pd.read_csv('data_files/scrape8.csv', index_col = 0)
df8 = pd.read_csv('data_files/scrape9.csv', index_col = 0)
df9 = pd.read_csv('data_files/scrape10.csv', index_col = 0)
df10 = pd.read_csv('data_files/scrape11.csv', index_col = 0)
df11 = pd.read_csv('data_files/scrape12.csv', index_col=0)
df12 = pd.read_csv('data_files/scrape13.csv', index_col=0)
df13 = pd.read_csv('data_files/scrape14.csv', index_col=0)
df14 = pd.read_csv('data_files/scrape15.csv', index_col=0)
df15 = pd.read_csv('data_files/scrape16.csv', index_col=0)
df16 = pd.read_csv('data_files/scrape17.csv', index_col=0)
df17 = pd.read_csv('data_files/scrape18.csv', index_col=0)

In [3]:
df = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17], axis=0)

In [4]:
df = df.reset_index(drop=True)

In [5]:
df.shape

(203126, 13)

In [8]:
df.sample(10)

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url
172401,4.4%,Year-round,3.53,NaN,Tirril Brewery,https://cdn.beeradvocate.com/im/placeholder-be...,England,Red Barn Ale,1,0,['3.53/5\xa0\xa0rDev 0%look: 4 | smell: 3.5 | ...,English Strong Ale,https://www.beeradvocate.com/beer/profile/3460...
60199,6.7%,Rotating,3.39,NaN,Fountainhead Brewing Company,https://cdn.beeradvocate.com/im/placeholder-be...,California,Fuzzy Notion,1,0,['3.39/5\xa0\xa0rDev 0%look: 3.5 | smell: 3.25...,American IPA,https://www.beeradvocate.com/beer/profile/5352...
44000,6.2%,Rotating,3.91,NaN,Fairfield Craft Ales,https://cdn.beeradvocate.com/im/placeholder-be...,Connecticut,Little Darlin IPA,4,0,['3.93/5\xa0\xa0rDev +0.5%look: 4.25 | smell: ...,American IPA,https://www.beeradvocate.com/beer/profile/4337...
8942,5.5%,Rotating,0.00,NaN,Benelux Brewpub & Café,https://cdn.beeradvocate.com/im/placeholder-be...,Quebec,L'ouvrière,0,0,[],American Brown Ale,https://www.beeradvocate.com/beer/profile/1380...
161056,10.5%,Fall,4.00,NaN,Brasserie De Metropole,https://cdn.beeradvocate.com/im/beers/114201.jpg,Russian Federation,Last Punch,1,0,"['4/5\xa0\xa0rDev 0%Gakusei, Feb 27, 2014']",Russian Imperial Stout,https://www.beeradvocate.com/beer/profile/2330...
27216,4.4%,Year-round,3.80,NaN,Siebensternbräu,https://cdn.beeradvocate.com/im/beers/30530.jpg,Austria,7 Stern Prager Dunkles,9,3,['4.03/5\xa0\xa0rDev +6.1%look: 4.5 | smell: 4...,Munich Dunkel Lager,https://www.beeradvocate.com/beer/profile/4098...
150655,6.2%,Rotating,3.78,NaN,Lariat Lodge Brewing Company,https://cdn.beeradvocate.com/im/placeholder-be...,Colorado,Squaw Pass Stout,1,0,['3.78/5\xa0\xa0rDev 0%look: 4.25 | smell: 3.7...,American Stout,https://www.beeradvocate.com/beer/profile/4321...
19346,4.7%,Year-round,3.13,NaN,BrewPub København,https://cdn.beeradvocate.com/im/placeholder-be...,Denmark,William Wallace,2,0,['3.25/5\xa0\xa0rDev +3.8%look: 3.25 | smell: ...,Scottish Ale,https://www.beeradvocate.com/beer/profile/1209...
36516,4%,Rotating,3.74,86.0,Fourpure Brewing Co.,https://cdn.beeradvocate.com/im/beers/340255.jpg,England,Easy Peeler,16,9,['3.68/5\xa0\xa0rDev -1.6%look: 3.5 | smell: 3...,American IPA,https://www.beeradvocate.com/beer/profile/3336...
37261,6.6%,Year-round,3.72,86.0,SeaQuake Brewing,https://cdn.beeradvocate.com/im/beers/274702.jpg,California,Citra IPA,13,6,['2.82/5\xa0\xa0rDev -24.2%look: 3.5 | smell: ...,American IPA,https://www.beeradvocate.com/beer/profile/4670...


In [10]:
len(df.loc[8942, 'review'])

2

In [15]:
def get_avg_taste(string):
    """takes in the review and pulls out the rating for the taste and returns an average"""
    list_list = [re.findall("\d{1}\.*\d*", item) for item in re.findall("taste: \d{1}\.*\d*", string)]
    sum_feel = 0
    tot = len(list_list)
    for list_item in list_list:
        for item in list_item:
            sum_feel += float(item)
    if not tot == 0:
        return sum_feel/tot
    else:
        return None


def get_avg_look(string):
    list_list = [re.findall("\d{1}\.*\d*", item) for item in re.findall("look: \d{1}\.*\d*", string)]
    sum_feel = 0
    tot = len(list_list)
    for list_item in list_list:
        for item in list_item:
            sum_feel += float(item)
    if not tot == 0:
        return sum_feel/tot
    else:
        return None

def get_avg_smell(string):
    list_list = [re.findall("\d{1}\.*\d*", item) for item in re.findall("smell: \d{1}\.*\d*", string)]
    sum_feel = 0
    tot = len(list_list)
    for list_item in list_list:
        for item in list_item:
            sum_feel += float(item)
    if not tot == 0:
        return sum_feel/tot
    else:
        return None


def get_avg_feel(string):
    list_list = [re.findall("\d{1}\.*\d*", item) for item in re.findall("feel: \d{1}\.*\d*", string)]
    sum_feel = 0
    tot = len(list_list)
    for list_item in list_list:
        for item in list_item:
            if item == '3..25':
                sum_feel += 3.25
            else:
                sum_feel += float(item)
    if not tot == 0:
        return sum_feel/tot
    else:
        return None

In [16]:
df['taste_avg'] = df['review'].apply(get_avg_taste)
df['look_avg'] = df['review'].apply(get_avg_look)
df['smell_avg'] = df['review'].apply(get_avg_smell)
df['feel_avg'] = df['review'].apply(get_avg_feel)

In [17]:
df.head()

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url,taste_avg,look_avg,smell_avg,feel_avg
0,5.6%,Winter,3.60,82.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Winter Lager,"6,210","2,048",['3.31/5\xa0\xa0rDev -8.1%look: 4 | smell: 3.5...,German Bock,https://www.beeradvocate.com/beer/profile/35/101/,3.56,3.76,3.48,3.63
1,4.4%,Year-round,3.19,73.0,Spoetzl Brewery,https://cdn.beeradvocate.com/im/beers/101.jpg,Texas,Shiner Bock,"4,331","1,163","[""3.19/5\xa0\xa0rDev 0%look: 3.75 | smell: 2.7...",German Bock,https://www.beeradvocate.com/beer/profile/143/...,3.47,3.76,3.31,3.43
2,5.8%,Rotating,3.66,83.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Chocolate Bock,"3,101","1,225",['3.78/5\xa0\xa0rDev +3.3%look: 3.75 | smell: ...,German Bock,https://www.beeradvocate.com/beer/profile/35/1...,3.65,3.64,3.59,3.61
3,5.2%,Year-round,2.89,68.0,Anheuser-Busch,https://cdn.beeradvocate.com/im/beers/101.jpg,Missouri,Michelob AmberBock,"1,906",707,"[""3.09/5\xa0\xa0rDev +6.9%look: 3.5 | smell: 2...",German Bock,https://www.beeradvocate.com/beer/profile/29/1...,3.24,3.62,3.27,3.21
4,5.8%,Rotating,3.50,80.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Cherry Chocolate Bock,873,187,['3/5\xa0\xa0rDev -14.3%look: 3 | smell: 3 | t...,German Bock,https://www.beeradvocate.com/beer/profile/35/6...,3.28,3.69,3.62,3.43


In [18]:
def clean_review(review):
    """takes in a review and does the following:
    1. removes the '\xa0'
    2. removes the '\n'
    3. performs the simple preprocess from gensim"""
    review = review.replace(u'\\xa0', '')
    review = review.replace('\\n', '')
    review = gensim.utils.simple_preprocess(str(review), deacc=True)
    review = [word for word in review if word not in stop_words]
    meta_lemmed = [lemmatizer.lemmatize(word) for word in review]
#     c = " ".join(str(x) for x in meta_lemmed)
#     cleaned.append(c)
    return meta_lemmed

In [10]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [8]:
lemmatizer = WordNetLemmatizer()

In [11]:
stop_words.extend(['look', 'smell', 'taste', 'feel', 'overall', 'rdev', 'beer', 'beers','nice','adams', 'jan', 'feb', 'mar',
                  'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'ml', 'oz', 'date'])

In [25]:
df['clean_review'] = df['review'].apply(clean_review)

In [27]:
df.head()

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url,taste_avg,look_avg,smell_avg,feel_avg,clean_review
0,5.6%,Winter,3.60,82.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Winter Lager,"6,210","2,048",['3.31/5\xa0\xa0rDev -8.1%look: 4 | smell: 3.5...,German Bock,https://www.beeradvocate.com/beer/profile/35/101/,3.56,3.76,3.48,3.63,"[alright, one, depends, great, deep, copper, m..."
1,4.4%,Year-round,3.19,73.0,Spoetzl Brewery,https://cdn.beeradvocate.com/im/beers/101.jpg,Texas,Shiner Bock,"4,331","1,163","[""3.19/5\xa0\xa0rDev 0%look: 3.75 | smell: 2.7...",German Bock,https://www.beeradvocate.com/beer/profile/143/...,3.47,3.76,3.31,3.43,"[first, sampled, visited, father, texas, earli..."
2,5.8%,Rotating,3.66,83.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Chocolate Bock,"3,101","1,225",['3.78/5\xa0\xa0rDev +3.3%look: 3.75 | smell: ...,German Bock,https://www.beeradvocate.com/beer/profile/35/1...,3.65,3.64,3.59,3.61,"[black, color, little, light, penetration, fin..."
3,5.2%,Year-round,2.89,68.0,Anheuser-Busch,https://cdn.beeradvocate.com/im/beers/101.jpg,Missouri,Michelob AmberBock,"1,906",707,"[""3.09/5\xa0\xa0rDev +6.9%look: 3.5 | smell: 2...",German Bock,https://www.beeradvocate.com/beer/profile/29/1...,3.24,3.62,3.27,3.21,"[bottle, abv, purchased, cold, single, poured,..."
4,5.8%,Rotating,3.50,80.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Cherry Chocolate Bock,873,187,['3/5\xa0\xa0rDev -14.3%look: 3 | smell: 3 | t...,German Bock,https://www.beeradvocate.com/beer/profile/35/6...,3.28,3.69,3.62,3.43,"[initial, whiff, dr, pepper, cherry, dy, flavo..."


In [28]:
 # I might need to drop short reviews - do this on the cleaned reviews.
def drop_short_reviews(review):
    if len(review) < 10:
        return None
    else:
        return review

In [29]:
df['clean_review'] = df['clean_review'].apply(drop_short_reviews)

In [30]:
df['clean_review'].isna().sum()

93618

In [31]:
df.dropna(subset=['clean_review'], inplace=True)

In [32]:
df.shape

(109508, 18)

In [33]:
def to_string(list_):
    return " ".join(list_)

In [34]:
df['key_words'] = df['clean_review'].apply(to_string)

In [35]:
df.to_csv('full_no_null.csv')

# Here I apply my basic rec to the full dataset

In [3]:
df = pd.read_csv('full_no_null.csv', index_col = 0)

In [5]:
df.head()

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url,taste_avg,look_avg,smell_avg,feel_avg,clean_review,key_words
0,5.6%,Winter,3.60,82.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Winter Lager,"6,210","2,048",['3.31/5\xa0\xa0rDev -8.1%look: 4 | smell: 3.5...,German Bock,https://www.beeradvocate.com/beer/profile/35/101/,3.56,3.76,3.48,3.63,"['alright', 'one', 'depends', 'great', 'deep',...",alright one depends great deep copper molasses...
1,4.4%,Year-round,3.19,73.0,Spoetzl Brewery,https://cdn.beeradvocate.com/im/beers/101.jpg,Texas,Shiner Bock,"4,331","1,163","[""3.19/5\xa0\xa0rDev 0%look: 3.75 | smell: 2.7...",German Bock,https://www.beeradvocate.com/beer/profile/143/...,3.47,3.76,3.31,3.43,"['first', 'sampled', 'visited', 'father', 'tex...",first sampled visited father texas earlier yea...
2,5.8%,Rotating,3.66,83.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Chocolate Bock,"3,101","1,225",['3.78/5\xa0\xa0rDev +3.3%look: 3.75 | smell: ...,German Bock,https://www.beeradvocate.com/beer/profile/35/1...,3.65,3.64,3.59,3.61,"['black', 'color', 'little', 'light', 'penetra...",black color little light penetration fingernai...
3,5.2%,Year-round,2.89,68.0,Anheuser-Busch,https://cdn.beeradvocate.com/im/beers/101.jpg,Missouri,Michelob AmberBock,"1,906",707,"[""3.09/5\xa0\xa0rDev +6.9%look: 3.5 | smell: 2...",German Bock,https://www.beeradvocate.com/beer/profile/29/1...,3.24,3.62,3.27,3.21,"['bottle', 'abv', 'purchased', 'cold', 'single...",bottle abv purchased cold single poured fluted...
4,5.8%,Rotating,3.50,80.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Cherry Chocolate Bock,873,187,['3/5\xa0\xa0rDev -14.3%look: 3 | smell: 3 | t...,German Bock,https://www.beeradvocate.com/beer/profile/35/6...,3.28,3.69,3.62,3.43,"['initial', 'whiff', 'dr', 'pepper', 'cherry',...",initial whiff dr pepper cherry dy flavor give ...


In [4]:
df.describe()

,avg_score,ba_score,taste_avg,look_avg,smell_avg,feel_avg
count,109508.000000,41411.000000,108968.000000,108968.000000,108968.000000,108968.000000
mean,3.756687,85.116853,3.762992,3.827705,3.737850,3.748186
std,0.429753,5.716217,0.519760,0.439248,0.491403,0.491963
min,1.000000,42.000000,1.000000,1.000000,1.000000,1.000000
25%,3.560000,82.000000,3.500000,3.625000,3.500000,3.500000
50%,3.810000,86.000000,3.812500,3.890000,3.750000,3.778846
75%,4.020000,88.000000,4.060000,4.040000,4.000000,4.000000
max,5.000000,100.000000,5.000000,5.000000,5.000000,5.000000


In [23]:
df.drop(['abv', 'avail', 'avg_score', 'ba_score', 'brewery', 'img', 'location',
         'no_ratings', 'no_reviews', 'review', 'style', 'url',
       'taste_avg', 'look_avg', 'smell_avg', 'feel_avg', 'clean_review'], inplace=True, axis=1)

In [24]:
df.set_index(['name'], inplace=True, drop=True)

In [28]:
df.sample(10)

,key_words
name,
Rotterdams Stoombier,copper orange color small headsmell corky slig...
Lighter Than I Look,solid dark lager rather traditional maybe touc...
Pre-prohibition Lager,solid bit odd smelling irony aftertaste color ...
Pommy Bitter,dansmcd pommy ok scent bolstered fuggle hop gi...
Blue Hulk,pours clear golden color head finger white fro...
Lösch-Zwerg Radler,pale yellow straw large white head good lacing...
Quadrupel,undated large bottle least year old poured dar...
Sangria,cloudy reddish amber light pink head acing gre...
Simple Needs,gold color excellent clarity vibrant carbonati...


In [38]:
stop_words.extend(['alright', 'one', 'pour', 'pours', 'gluten', 'made', 'glass', 'one', 'bit', 'maybe', 'pretty',
                  'served', 'key', 'keg', 'good', 'wonder', 'mix', 'left', 'inch', 'american', 'style', 'similar'])

In [36]:
FreqDist(df['key_words']).most_common(50)

[('made especially independent salford festival revenge fifth brewed keg served via key keg retains real ale marker helmsworth house',
  16),
 ('ruddy copper colored slightly white head strong aroma caramel taste caramel butterscotch coffee earth roasted nut sweet nutty finish touch bitter coffee like caramel coffee mix wonder bit lactic cloying work quite nicely',
  3),
 ('growler georgetown brewing taproom poured wine glass tyellowish pale hazy medium white foam tvery fragrant hoppy tmedium bitterness lively carbonation toverall good excellent value ok repeat fordrank march',
  3),
 ('tried brewery whilst town work conference poured slightly hazy orangish thin white head leave much lacing scent citrus fruit note nicely balanced easy drink malt base slight bitter fruit finish mouthfeel lighter body good carbonation solid',
  3),
 ('pours hazy reddish color carbonation one finger plus white head sticky lacing left behind nose malty little hop bite caramel note malty hop note bitter dry

In [31]:
df.loc['Pommy Bitter', 'key_words']

'dansmcd pommy ok scent bolstered fuggle hop give floral note like juicy fruit gum worth try'

In [21]:
df.loc[0]

abv                                                          5.6%
avail                                                      Winter
avg_score                                                     3.6
ba_score                                                       82
brewery                        Boston Beer Company (Samuel Adams)
img                 https://cdn.beeradvocate.com/im/beers/101.jpg
location                                            Massachusetts
name                                    Samuel Adams Winter Lager
no_ratings                                                  6,210
no_reviews                                                  2,048
review          ['3.31/5\xa0\xa0rDev -8.1%look: 4 | smell: 3.5...
style                                                 German Bock
url             https://www.beeradvocate.com/beer/profile/35/101/
taste_avg                                                    3.56
look_avg                                                     3.76
smell_avg 

In [39]:
# instantiating and generating the count matrix
count = CountVectorizer(stop_words=stop_words, max_df=0.7, min_df=.1)
count_matrix = count.fit_transform(df['key_words'])
count_matrix.shape
# try min_df or max_df

(109508, 368)

In [40]:
import pickle
pickle.dump(count_matrix, open('count_matrix_tiny.p', 'wb'))

In [16]:
# or we could do tfidf:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2),max_df=.8, min_df=5, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(df['key_words'])
tfidf_matrix.shape

(109508, 723593)

In [ ]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)

#  defining the function that takes in beer name 
# as input and returns the top 10 recommended beers
def recommendations(cosine_sim = cosine_sim):
    title = input('What beer do you like?')
    # initializing the empty list of recommended beers
    recommended_beers = []
    
    # gettin the index of the beer that matches the name
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching beers
    for i in top_10_indexes:
        recommended_beers.append(list(df.index)[i])
        
    return recommended_beers